In [1]:
!pip install fuzzywuzzy



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.metrics import accuracy_score
from fuzzywuzzy import process  # NLP spelling correction

# Load dataset
df = pd.read_csv("C:/Users/muthu/Downloads/regionally_valid_diet_meal_plan (1).csv")

# Updated input columns including 'Region'
input_cols = ['Age', 'Gender', 'BMI', 'Activity Level', 'Goal', 'Medical Conditions', 'Allergies', 'Dietary Preference', 'Region']
meal_cols = ['Breakfast', 'Lunch', 'Dinner', 'Snack 1', 'Snack 2']

# Encode categorical columns for input
le_dict = {}
for col in input_cols:
    if df[col].dtype == 'object':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        le_dict[col] = le

# Combine all meals into one string
df['Meal Plan'] = df[meal_cols].agg('|'.join, axis=1)
meal_encoder = LabelEncoder()
df['Meal Plan Encoded'] = meal_encoder.fit_transform(df['Meal Plan'])

# Q-Learning Setup
states = df[input_cols].astype(str).agg('-'.join, axis=1).values
actions = df['Meal Plan Encoded'].values

# Initialize Q-table
Q = defaultdict(lambda: np.zeros(len(meal_encoder.classes_)))

# Q-Learning Parameters
epsilon = 0.1  # Exploration rate
alpha = 0.5    # Learning rate
gamma = 0.9    # Discount factor
epochs = 10

# Training loop
for _ in range(epochs):
    for i in range(len(states)):
        state = states[i]
        action = actions[i]
        reward = 1  # Static reward; can be made dynamic

        if random.uniform(0, 1) < epsilon:
            next_action = random.randint(0, len(meal_encoder.classes_) - 1)
        else:
            next_action = np.argmax(Q[state])

        Q[state][action] += alpha * (reward + gamma * np.max(Q[state]) - Q[state][action])

# Inference function with fuzzy NLP correction
def suggest_meal():
    print("\nEnter user details:")
    user_input = {}

    for col in input_cols:
        value = input(f"{col}: ")
        if col in le_dict:
            known_values = list(le_dict[col].classes_)
            best_match, score = process.extractOne(value, known_values)
            if score >= 80:  # Fuzzy threshold
                print(f"Interpreted '{value}' as '{best_match}' for column '{col}'")
                value = best_match
                value = le_dict[col].transform([value])[0]
            else:
                print(f"Value '{value}' not recognized for {col}. Try values like: {known_values}")
                return
        else:
            try:
                value = float(value)
            except ValueError:
                print(f"Invalid numeric input for {col}")
                return
        user_input[col] = value

    state_str = '-'.join([str(user_input[col]) for col in input_cols])

    if state_str not in Q:
        print("No matching state found. Suggestion may not be optimal.")
        best_action = random.randint(0, len(meal_encoder.classes_) - 1)
    else:
        best_action = np.argmax(Q[state_str])

    meal_plan = meal_encoder.inverse_transform([best_action])[0]
    breakfast, lunch, dinner, snack1, snack2 = meal_plan.split('|')
    return {
        'Breakfast': breakfast,
        'Lunch': lunch,
        'Dinner': dinner,
        'Snack 1': snack1,
        'Snack 2': snack2
    }

# Evaluate model accuracy
y_true = actions
y_pred = [np.argmax(Q[state]) for state in states]
accuracy = accuracy_score(y_true, y_pred)
print(f"\nModel Accuracy: {accuracy * 100:.2f}%")

# Interactive suggestion
result = suggest_meal()
if result:
    print("\nSuggested Meal Plan:", result)


c:\Users\muthu\AppData\Local\Programs\Python\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



Model Accuracy: 100.00%

Enter user details:
Interpreted 'Male' as 'Male' for column 'Gender'
Interpreted 'Very active' as 'Very active' for column 'Activity Level'
Interpreted 'Weight Loss' as 'Weight Loss' for column 'Goal'
Interpreted 'Hypertension' as 'Hypertension' for column 'Medical Conditions'
Interpreted 'Dairy' as 'Dairy' for column 'Allergies'
Interpreted 'Vegetarian' as 'Vegetarian' for column 'Dietary Preference'
Interpreted 'Bihar' as 'Bihar' for column 'Region'
No matching state found. Suggestion may not be optimal.

Suggested Meal Plan: {'Breakfast': 'Vegetable sandwich', 'Lunch': 'Spinach and corn pasta', 'Dinner': 'Dal with rice', 'Snack 1': 'Carrot sticks with hummus', 'Snack 2': 'Trail mix'}
